**Условие задачи**

В архиве содержится три текстовых файла:

qid_query.tsv — id запроса и текст запроса, разделённые табуляцией;

qid_url_rating.tsv — id запроса, URL документа, релевантность документа запросу;

hostid_url.tsv — id хоста и URL документа.

Нужно вывести текст запроса с максимальным значением метрики
pFound
, посчитанной по топ-10 документов.
Если для запроса есть несколько документов с одним и тем же id хоста — оставить только максимально релевантный документ (а если несколько документов максимально релевантны, выбрать любой).
Документы по запросу сортируются по убыванию релевантности после выбора одного документа для хоста. Если у нескольких документов с разных хостов релевантность одинакова, их порядок может быть произвольным.

**ФОРМАТ ВЫВОДА**

Текст запроса с максимальным значением метрики.

In [86]:
import pandas as pd

In [87]:
hostid = pd.read_csv('hostid_url.tsv', sep='\t', header=None)

In [88]:
hostid = hostid.rename({0:'id_host', 1:'URL'}, axis=1)

In [89]:
hostid.head()

,id_host,URL
0,100,http://cons66.ru/
1,1000,http://www.pozdravleniya.biz/stihi/detskie/
2,1001,http://www.pravda.ru/society/family/pbringing/...
3,1002,http://www.pravpiter.ru/gorn/n015/ta003.htm
4,1003,http://www.pregnancycalendar.ru/biblioteka/vse...


In [90]:
hostid.shape

(854, 2)

In [91]:
hostid['id_host'].unique().shape

(643,)

In [92]:
qid_query = pd.read_csv('qid_query.tsv', sep='\t', header=None)

In [93]:
qid_query = qid_query.rename({0:'id_q', 1:'text'}, axis=1)

In [94]:
qid_query.head()

,id_q,text
0,10387,batman arkham city будет русский перевод?
1,107538,Список белорусской литературы по лизингу
2,150126,написание review
3,168170,медаль в подарок на 30 лет
4,176370,что означает рожь и пшеница годом родится а ве...


In [95]:
qid_query.shape

(20, 2)

In [96]:
qid_url_rating = pd.read_csv('qid_url_rating.tsv', sep='\t', header=None)

In [97]:
qid_url_rating = qid_url_rating.rename({0:'id_q', 1:'URL', 2:'relevance'}, axis=1)

In [98]:
qid_url_rating.head()

,id_q,URL,relevance
0,10387,http://batman-arkhamcity.ru/,0.00
1,10387,http://bigtorrents.org/publ/batman_arkham_city...,0.14
2,10387,http://consolelife.ru/xbox-360/6577-o-rossiysk...,0.14
3,10387,http://dic.academic.ru/book.nsf/3662736/Batman...,0.00
4,10387,http://forum.csmania.ru/viewtopic.php?t=25986,0.14


In [99]:
qid_url_rating.shape

(854, 3)

In [100]:
hostid.shape

(854, 2)

In [101]:
qid_url_rating_hostid = hostid.merge(qid_url_rating, on="URL")

In [102]:
qid_url_rating_hostid.head()

,id_host,URL,id_q,relevance
0,100,http://cons66.ru/,347852,0.00
1,1000,http://www.pozdravleniya.biz/stihi/detskie/,242953,0.14
2,1001,http://www.pravda.ru/society/family/pbringing/...,221830,0.07
3,1002,http://www.pravpiter.ru/gorn/n015/ta003.htm,366042,0.00
4,1003,http://www.pregnancycalendar.ru/biblioteka/vse...,346214,0.00


In [103]:
def plook(ind, rels):
	    return 1 if ind == 0 else plook(ind-1, rels)*(1-rels[ind-1])*(1-0.15)

pFound = ∑_i (i from 1 to 10) pLook [ 𝑖 ] · pRel[i]


pLook[1] = 1

pLook [ 𝑖 ] = pLook [ 𝑖 − 1 ] · ( 1 − pRel [ 𝑖 − 1 ] ) · ( 1 − pBreak )

pBreak = 0.15

In [104]:
def pfound(group):
  max_by_host = group.groupby("id_host")["relevance"].max() # максимальный рейтинг хоста
  top10 = max_by_host.sort_values(ascending=False)[:10] # берем топ10 урлов с наивысшим рейтингом
  pfound = 0
  for ind, val in enumerate(top10):
      pfound += val*plook(ind, top10.values)
  return pfound

In [105]:
qid_pfound = qid_url_rating_hostid.groupby('id_q').apply(pfound) # группируем по qid и вычисляем pfound
qid_max = qid_pfound.idxmax() # берем qid с максимальным pfound

<ipython-input-105-b260e2e4e3eb>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  qid_pfound = qid_url_rating_hostid.groupby('id_q').apply(pfound) # группируем по qid и вычисляем pfound


In [107]:
qid_query[qid_query["id_q"] == qid_max]

,id_q,text
12,295761,гугл переводчик


**Задание 2**